#### Session 4 Homework

In [1]:
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import glob
%matplotlib inline
def plot_corr(df,cols):
    fig = plt.figure(figsize=(10,8))
    sns.heatmap(df[cols].corr(),cmap='RdBu_r', annot=True, center=0.0)
    plt.show()
    
def plot_count(df,col,fsize,rotation=None,fillna=True):
    fig = plt.figure(figsize=fsize)
    if fillna:
        value_count = df[col].fillna('unknown').value_counts()
    sns.barplot(value_count.index,value_count.values)
    plt.xticks(rotation=rotation)
    plt.title('value counts for {}'.format(col))
    plt.show()

### Assignment 1

In [2]:
DATA_PATH = r'C:\\Users\\new-user-bookings\\'
file_names = glob.glob(os.path.join(DATA_PATH + "*.csv"))

#Function to load data
def load_data(path):
    file_names = glob.glob(os.path.join(path + "*.csv"))
    return file_names
load_data(DATA_PATH)

['C:\\\\Users\\\\new-user-bookings\\age_gender_bkts.csv',
 'C:\\\\Users\\\\new-user-bookings\\countries.csv',
 'C:\\\\Users\\\\new-user-bookings\\sessions.csv',
 'C:\\\\Users\\\\new-user-bookings\\train_users_2.csv']

### Assignment 2

In [3]:
train_df = pd.read_csv(file_names[3])
sessions_df = pd.read_csv(file_names[2])

In [4]:
#Convert types
train_df['timestamp_first_active'] = train_df['timestamp_first_active'].astype(np.int32)
train_df['age'] = train_df['age'].astype(np.float16)
train_df['signup_flow'] = train_df['signup_flow'].astype(np.int16)
sessions_df['secs_elapsed'] = sessions_df['secs_elapsed'].astype(np.float32)

### Assignment 3

In [5]:
nan_info_train = pd.DataFrame(train_df.isnull().sum()).reset_index()
nan_info_train.columns = ['col','nan_cnt']
nan_info_train.sort_values(by = 'nan_cnt',ascending=False,inplace=True)
nan_info_train

,col,nan_cnt
3,date_first_booking,124543
5,age,87990
11,first_affiliate_tracked,6065
0,id,0
1,date_account_created,0
2,timestamp_first_active,0
4,gender,0
6,signup_method,0
7,signup_flow,0
8,language,0


In [7]:
nan_info_sessions = pd.DataFrame(sessions_df.isnull().sum()).reset_index()
nan_info_sessions.columns = ['col','nan_cnt']
nan_info_sessions.sort_values(by = 'nan_cnt',ascending=False,inplace=True)
nan_info_sessions

,col,nan_cnt
2,action_type,1126204
3,action_detail,1126204
5,secs_elapsed,136031
1,action,79626
0,user_id,34496
4,device_type,0


In [8]:
train_cols_with_missing = nan_info_train.loc[nan_info_train.nan_cnt>0].col.values
train_cols_with_missing

array(['date_first_booking', 'age', 'first_affiliate_tracked'],
      dtype=object)

In [9]:
sessions_cols_with_missing = nan_info_sessions.loc[nan_info_sessions.nan_cnt>0].col.values
sessions_cols_with_missing

array(['action_type', 'action_detail', 'secs_elapsed', 'action',
       'user_id'], dtype=object)

In [10]:
for f in train_cols_with_missing:
    print(f, train_df[f].dtype)

date_first_booking object
age float16
first_affiliate_tracked object


In [11]:
for f in sessions_cols_with_missing:
    print(f, sessions_df[f].dtype)

action_type object
action_detail object
secs_elapsed float32
action object
user_id object


In [12]:
train_df['age'].fillna(train_df['age'].median(), inplace=True)
sessions_df['secs_elapsed'].fillna(sessions_df['secs_elapsed'].mean(), inplace=True)

### Assignment 4

In [13]:
#Subset 1
train_df['gender'].unique()

array(['-unknown-', 'MALE', 'FEMALE', 'OTHER'], dtype=object)

In [14]:
temp_df = train_df.loc[train_df['gender'] == 'MALE']
temp_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
1,820tgsjxq7,2011-05-25,-1333835879,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
21,qsibmuz9sx,2010-01-10,-336724339,2010-01-11,MALE,30.0,basic,0,en,direct,direct,linked,Web,Mac Desktop,Chrome,US
31,pggg2sj27u,2010-01-14,-332884523,NaN,MALE,31.0,basic,0,en,direct,direct,NaN,Web,Other/Unknown,-unknown-,NDF
33,qthj88nnc7,2010-01-14,-332770071,NaN,MALE,29.0,basic,0,en,other,craigslist,NaN,Web,Other/Unknown,-unknown-,NDF
34,x9gnxoun57,2010-01-15,-331890227,NaN,MALE,59.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,NDF


In [15]:
#Subset 2
temp_df = train_df.loc[train_df['age'] > 18 ]
temp_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,-1537967433,NaN,-unknown-,34.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,-1333835879,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,-1247779441,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,-825950559,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,-648949583,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [16]:
#Subset 3
train_df['language'].unique()

array(['en', 'fr', 'de', 'es', 'it', 'pt', 'zh', 'ko', 'ja', 'ru', 'pl',
       'el', 'sv', 'nl', 'hu', 'da', 'id', 'fi', 'no', 'tr', 'th', 'cs',
       'hr', 'ca', 'is'], dtype=object)

In [17]:
temp_df = train_df.loc[train_df['language'] == 'en' ]
temp_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,-1537967433,NaN,-unknown-,34.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,-1333835879,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,-1247779441,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,-825950559,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,-648949583,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [18]:
#Subset 4
train_df['affiliate_provider'].unique()

array(['direct', 'google', 'other', 'craigslist', 'facebook', 'vast',
       'bing', 'meetup', 'facebook-open-graph', 'email-marketing',
       'yahoo', 'padmapper', 'gsp', 'wayn', 'naver', 'baidu', 'yandex',
       'daum'], dtype=object)

In [19]:
temp_df = train_df.loc[train_df['affiliate_provider'] == 'direct' ]
temp_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,-1537967433,NaN,-unknown-,34.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,-1247779441,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,-825950559,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,-648949583,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
7,0d01nltbrs,2010-01-03,-343753375,2010-01-13,FEMALE,47.0,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US


In [20]:
#Subset 5
sessions_df['action'].unique()

array(['lookup', 'search_results', 'personalize', 'index',
       'similar_listings', 'ajax_refresh_subtotal', 'show',
       'header_userpic', 'ask_question', nan,
       'other_hosting_reviews_first', 'hosting_social_proof',
       'decision_tree', 'recent_reservations', 'faq_experiment_ids',
       'multi', 'active', 'dashboard', 'create', 'confirm_email',
       'show_personalize', 'verify', 'pending', 'requested', 'concierge',
       'faq', 'clear_reservation', 'cancellation_policies',
       'track_page_view', 'update', 'my', 'campaigns', 'notifications',
       'listings', 'unavailabilities', 'ajax_lwlb_contact',
       'ajax_check_dates', 'qt2', 'request_new_confirm_email',
       'ajax_photo_widget_form_iframe', 'facebook_auto_login', 'identity',
       'qt_reply_v2', 'travel_plans_current', 'complete_status',
       'populate_from_facebook', 'kba_update', 'kba', 'login',
       'authenticate', 'calendar_tab_inner2', 'other_hosting_reviews',
       'social_connections', 'relat

In [21]:
temp_df = sessions_df.loc[sessions_df['action'] == 'lookup']
temp_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0
6,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,115.0
9,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,683.0


### Assignment 5

In [22]:
#Feature Aggregation Categorical
cat_agg_dict = {'action':['nunique'], 'action_detail':['count']}
cat_agg_df = sessions_df.rename(columns={'user_id':'id'}).groupby('id').agg(cat_agg_dict)
cat_agg_df.columns = ['{}_{}'.format(x[0],x[1]) for x in cat_agg_df.columns.tolist()]
cat_agg_df.head()

,action_nunique,action_detail_count
id,,
00023iyk9l,13,37
0010k6l0om,11,48
001wyh0pz8,10,85
0028jgx1x1,5,31
002qnbzfs5,25,712


In [23]:
#Feature Aggregation numeric
num_agg_dict = {'secs_elapsed':['mean','max','min','std','sum']}
num_agg_df = sessions_df.rename(columns={'user_id':'id'}).groupby('id').agg(num_agg_dict)
num_agg_df.columns = ['{}_{}'.format(x[0],x[1]) for x in num_agg_df.columns.tolist()]
num_agg_df.head()

,secs_elapsed_mean,secs_elapsed_max,secs_elapsed_min,secs_elapsed_std,secs_elapsed_sum
id,,,,,
00023iyk9l,22171.607422,567165.0,0.0,91053.765625,8.868643e+05
0010k6l0om,9611.291016,127898.0,3.0,22598.775391,6.055113e+05
001wyh0pz8,3354.814453,50548.0,35.0,6741.307129,3.019333e+05
0028jgx1x1,10192.848633,84636.0,3.0,17777.101562,3.159783e+05
002qnbzfs5,8245.942383,1418284.0,0.0,58075.019531,6.506048e+06


In [24]:
#Merge categorical data
mergeddata = train_df.merge(cat_agg_df,on='id',how='inner')
mergeddata.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,action_nunique,action_detail_count
0,d1mm9tcy42,2014-01-01,999349992,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Chrome,other,16,90
1,yo8nz8bqcq,2014-01-01,999350614,NaN,-unknown-,34.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF,7,8
2,4grx6yxeby,2014-01-01,999350695,NaN,-unknown-,34.0,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Firefox,NDF,12,16
3,ncf87guaf0,2014-01-01,999351202,NaN,-unknown-,34.0,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF,19,125
4,4rvqpxoh3h,2014-01-01,999351675,2014-01-02,-unknown-,34.0,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,GB,7,8


In [25]:
#Merge numeric data
mergeddata = train_df.merge(num_agg_df,on='id',how='inner')
mergeddata.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,secs_elapsed_mean,secs_elapsed_max,secs_elapsed_min,secs_elapsed_std,secs_elapsed_sum
0,d1mm9tcy42,2014-01-01,999349992,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,...,omg,Web,Windows Desktop,Chrome,other,27137.773438,606881.000000,2.0,69089.296875,3.446497e+06
1,yo8nz8bqcq,2014-01-01,999350614,NaN,-unknown-,34.0,basic,0,en,direct,...,untracked,Web,Mac Desktop,Firefox,NDF,25201.144531,115983.000000,36.0,37048.343750,2.268103e+05
2,4grx6yxeby,2014-01-01,999350695,NaN,-unknown-,34.0,basic,0,en,sem-brand,...,omg,Web,Windows Desktop,Firefox,NDF,72150.765625,336801.000000,53.0,90514.117188,1.154412e+06
3,ncf87guaf0,2014-01-01,999351202,NaN,-unknown-,34.0,basic,0,en,direct,...,linked,Web,Windows Desktop,Chrome,NDF,24829.396484,732296.000000,0.0,84117.289062,3.774068e+06
4,4rvqpxoh3h,2014-01-01,999351675,2014-01-02,-unknown-,34.0,basic,25,en,direct,...,untracked,iOS,iPhone,-unknown-,GB,2690.413330,18968.306641,21.0,6584.032715,2.152331e+04


### Week 4 Feature Engineering

In [26]:
sessions_df['secs_elapsed'].describe()

count    1.056774e+07
mean     1.894560e+04
std      8.697572e+04
min      0.000000e+00
25%      2.370000e+02
50%      1.188000e+03
75%      9.227000e+03
max      1.799977e+06
Name: secs_elapsed, dtype: float64

In [27]:
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [28]:
#Find the number of times each user has viewed search results
add_vsr = sessions_df.rename(columns={'user_id':'id'}).groupby('id')['action_detail'].agg(lambda x:list(x).count('view_search_results'))
add_vsr.sort_values()

id
gwxxyo0r2m      0
9ejtm687m6      0
pu77g3m78u      0
v7gb7omyik      0
9ekj8i1u47      0
s5y9r7nvrw      0
9elzvahosn      0
9em31ovenn      0
pu7gz6o1xl      0
v7fg1l1v9e      0
9en6741a79      0
v7f7hnzyli      0
v7f1m5pzch      0
v7e9l2d0zl      0
hiy5qkl8ym      0
k25zw94xjv      0
s5ylld7u75      0
9eoxx48xfs      0
9epid1ni1w      0
v7dh9rpb48      0
9eplss8mq0      0
9epmtc0epv      0
9ej7227172      0
pu8d71xkd0      0
9ehmdxhccw      0
9eh32mrgvh      0
9e8plbg0rr      0
9e8ynuiafs      0
pu2lu01z56      0
v7n94gnibq      0
             ... 
rcqg09gqwc    488
eqtvgzrz3n    491
gfoj3j7yhn    492
sedfvt0adf    495
uopnwd0cn1    501
kro83jmtt8    503
5lolu8ir9b    503
h1p33umgua    509
k3w4kskp4o    514
mbu0qnxb91    517
ppildsrvkl    542
y4fz9ytpz8    559
ak520g6ljp    562
jtuqmjx5j6    578
5vpuk5mssg    584
jwm6c71uhb    586
o0jceebxed    594
bmq1hoqurj    596
ljpf68lhes    614
2gymyppjlo    615
t9m35skje8    619
9z4gim1s4l    628
2ye81t5oh4    638
qitt3fvbkg    641
ytplei6

In [29]:
#Find the number of times each user has updated their wishlist
add_uwl = sessions_df.rename(columns={'user_id':'id'}).groupby('id')['action_detail'].agg(lambda x:list(x).count('wishlist_content_update'))
add_uwl.sort_values()

id
dqaxpikkbw      0
g9icvz2xej      0
uyf4mcejqp      0
uyfhwp9d10      0
g9fv3nszn0      0
uyg2fkqkp2      0
uygi2kex5w      0
g9egakm9x3      0
uygp63uao7      0
g9dglt5lwi      0
g9db8f9tl2      0
uyh3nje7oq      0
g9d4wvp0n3      0
g9d2nw19w2      0
g9c2jn01b7      0
uyhw66mnmd      0
g9il0l1o8z      0
uyeutclwpd      0
g9jh5xhzef      0
g9jrxt3xmv      0
uybztjszv4      0
uycfvzfcla      0
g9sa0l79qe      0
g9rtdu9976      0
uyd15m3zod      0
g9r5dzugop      0
uydz6bnrnd      0
g9aoibp6x3      0
g9q85ztnal      0
g9q1snv0f1      0
             ... 
njk1ykrn9t    191
5e4ai0ole5    192
d5i9x9ej04    193
fsa2gw8kk4    194
e81qfos71y    197
6nxatkk69j    201
jastc45uok    206
h0fd2340p6    207
d8cgiviyy9    208
bfiueza7rt    212
z3mgcgzfun    213
nq0c3g76js    216
pbajcbv3wx    223
n4s6g3grzf    223
g1nf1o6wku    227
6xx55fvyz7    229
e4kn8hbtui    230
j2cvctvqve    233
bp4mwnlbxs    244
tu41qoa28z    252
0kj6w88xa2    262
2xwvnz848a    265
ca8jmor156    266
b1io359wpg    297
ht8alhs

In [30]:
#Find the number of times each user has contacted the host
add_ch = sessions_df.rename(columns={'user_id':'id'}).groupby('id')['action_detail'].agg(lambda x:list(x).count('contact_host'))
add_ch.sort_values()

id
00023iyk9l      0
mk19r5ftze      0
mk19evjucm      0
mk0q3c5twq      0
mk0otithyh      0
mk0ck0h9zs      0
mjzefwuypi      0
mjzc0izmgs      0
mjz9msdkzr      0
mjyys0zlcb      0
mjy9va04op      0
mjxsz5rbv8      0
mjxjp9lpjl      0
mjxeh2rqtj      0
mjxaof4ydc      0
mjwfx44co4      0
mjw5a13vvv      0
mjw12yv4e1      0
mjv63mjdje      0
mjttr680je      0
mjsyx3jbv2      0
mjs23nmsa1      0
mjrmoikk5n      0
mjr5iv23a6      0
mjqtfj53s6      0
mjqez799h0      0
mk1aivd68r      0
mjp60z2hx8      0
mk1h0dxp8y      0
mk2jr969n4      0
             ... 
a0gpn5wseo     39
a5qdpu9nev     39
r8xsirhrjz     40
ubtc46x95z     41
2f8hejmd4q     41
g7v2pdhwr3     42
masrzmahd8     42
t6q9ckz69s     43
c4o8purxg9     45
3zmeajzkf1     45
lol90toh6v     46
k8r50gmj0u     47
mn8q9ua8ug     47
bhqkyggmqu     48
55hbmqaela     48
mcyg3w2645     48
jfmutojdae     50
6l6g69rf29     51
snui827tsu     53
lfr18hrckm     55
tndeov9ar7     58
61gpwqfixg     59
q5qcdbntwk     65
aqaf70t5o1     67
jrqykh9

In [31]:
nan_info = pd.DataFrame(sessions_df.isnull().sum()).reset_index()
nan_info.columns = ['col', 'nan_cnt']
cols_with_missing = nan_info.loc[nan_info.nan_cnt>0].col.values
sessions_df['action_detail'].fillna(value = 'unknown')

0                               unknown
1                   view_search_results
2                               unknown
3                   view_search_results
4                               unknown
5                   view_search_results
6                               unknown
7               wishlist_content_update
8                   view_search_results
9                               unknown
10                  view_search_results
11                              unknown
12              wishlist_content_update
13                  view_search_results
14                     similar_listings
15          change_trip_characteristics
16                     similar_listings
17          change_trip_characteristics
18                              unknown
19              wishlist_content_update
20                                   p3
21                              unknown
22                  view_search_results
23                              unknown
24                  view_search_results


In [32]:
#Find the time elapsed when viewing search results
vsr_temp = sessions_df[sessions_df['action_detail'].str.match('view_search_results', na = False)]
add_vsr_time = vsr_temp.rename(columns={'user_id':'id'}).groupby('id').mean()
add_vsr_time

,secs_elapsed
id,
00023iyk9l,10958.200195
0010k6l0om,7595.100098
001wyh0pz8,2406.000000
0028jgx1x1,22200.222656
002qnbzfs5,3947.335938
0035hobuyj,11204.334961
00389675gq,17828.927734
003iamz20l,16632.044922
0048rkdgb1,48092.750000


In [33]:
#Merge all new features
features = [add_vsr, add_uwl, add_ch, add_vsr_time]
for f in features:
    mergeddata = mergeddata.merge(f,on='id',how='inner')
mergeddata

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,country_destination,secs_elapsed_mean,secs_elapsed_max,secs_elapsed_min,secs_elapsed_std,secs_elapsed_sum,action_detail_x,action_detail_y,action_detail,secs_elapsed
0,d1mm9tcy42,2014-01-01,999349992,2014-01-04,MALE,62.0,basic,0,en,sem-non-brand,...,other,27137.773438,606881.0,2.0,69089.296875,3.446497e+06,23,25,1,72824.828125
1,ncf87guaf0,2014-01-01,999351202,NaN,-unknown-,34.0,basic,0,en,direct,...,NDF,24829.396484,732296.0,0.0,84117.289062,3.774068e+06,32,10,0,7375.687500
2,5f45ro5uzk,2014-01-01,999352591,NaN,-unknown-,34.0,basic,0,en,direct,...,NDF,13573.506836,117638.0,23.0,25566.091797,6.243813e+05,7,7,0,35833.855469
3,ro2stddszp,2014-01-01,999354559,2014-12-04,-unknown-,19.0,basic,0,en,sem-brand,...,other,30310.914062,658848.0,37.0,105295.843750,1.303369e+06,5,7,5,199193.796875
4,qtw88d9pbl,2014-01-01,999354893,NaN,MALE,25.0,basic,0,en,direct,...,NDF,7931.794922,485255.0,0.0,36501.316406,2.887173e+06,39,43,16,29946.050781
5,ucgks2fyez,2014-01-01,999359956,2014-12-18,-unknown-,76.0,basic,0,en,seo,...,other,24126.113281,671005.0,0.0,68248.835938,6.827690e+06,49,40,1,51337.570312
6,j30uqo74v6,2014-01-01,999359963,NaN,-unknown-,34.0,basic,0,en,direct,...,NDF,8643.281250,73598.0,5.0,16315.250977,7.173923e+05,20,12,1,22612.699219
7,jrqykh9y8x,2014-01-01,999360975,2014-04-19,FEMALE,27.0,facebook,0,zh,seo,...,FR,12627.019531,1470144.0,0.0,63081.210938,1.476098e+07,145,118,67,15824.903320
8,s9xrwtyzsq,2014-01-01,999361401,2014-01-02,MALE,30.0,basic,0,en,sem-brand,...,US,29780.716797,469171.0,0.0,76655.703125,2.173992e+06,4,9,0,15443.250000
9,mde5wnphhf,2014-01-01,999364556,NaN,-unknown-,34.0,basic,0,en,sem-brand,...,NDF,24943.683594,629658.0,48.0,103250.054688,9.229163e+05,7,5,0,13855.286133


### Assignment 6

In [34]:
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [35]:
#a - concat two features
sessions_df['user_detailed_action'] = sessions_df[['action','action_type']].apply(lambda row: '_'.join(row.values.astype(str)),axis=1)

In [36]:
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed,user_detailed_action
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0,lookup_nan
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0,search_results_click
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0,lookup_nan
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0,search_results_click
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0,lookup_nan


In [37]:
#b - lower case
sessions_df['device_type'].apply(lambda x: x.lower())

0           windows desktop
1           windows desktop
2           windows desktop
3           windows desktop
4           windows desktop
5           windows desktop
6           windows desktop
7           windows desktop
8           windows desktop
9           windows desktop
10          windows desktop
11          windows desktop
12          windows desktop
13          windows desktop
14          windows desktop
15          windows desktop
16          windows desktop
17          windows desktop
18          windows desktop
19          windows desktop
20          windows desktop
21          windows desktop
22          windows desktop
23          windows desktop
24          windows desktop
25          windows desktop
26          windows desktop
27          windows desktop
28          windows desktop
29          windows desktop
                 ...       
10567707        mac desktop
10567708        mac desktop
10567709        mac desktop
10567710        mac desktop
10567711        mac 

In [38]:
#bc - Upper case
sessions_df['device_type'].apply(lambda x: x.upper())

0           WINDOWS DESKTOP
1           WINDOWS DESKTOP
2           WINDOWS DESKTOP
3           WINDOWS DESKTOP
4           WINDOWS DESKTOP
5           WINDOWS DESKTOP
6           WINDOWS DESKTOP
7           WINDOWS DESKTOP
8           WINDOWS DESKTOP
9           WINDOWS DESKTOP
10          WINDOWS DESKTOP
11          WINDOWS DESKTOP
12          WINDOWS DESKTOP
13          WINDOWS DESKTOP
14          WINDOWS DESKTOP
15          WINDOWS DESKTOP
16          WINDOWS DESKTOP
17          WINDOWS DESKTOP
18          WINDOWS DESKTOP
19          WINDOWS DESKTOP
20          WINDOWS DESKTOP
21          WINDOWS DESKTOP
22          WINDOWS DESKTOP
23          WINDOWS DESKTOP
24          WINDOWS DESKTOP
25          WINDOWS DESKTOP
26          WINDOWS DESKTOP
27          WINDOWS DESKTOP
28          WINDOWS DESKTOP
29          WINDOWS DESKTOP
                 ...       
10567707        MAC DESKTOP
10567708        MAC DESKTOP
10567709        MAC DESKTOP
10567710        MAC DESKTOP
10567711        MAC 

### Assignment 7

In [39]:
# Hot encoding 1

cat_features = ['gender', 'signup_method', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app','first_device_type', 'first_browser', 'country_destination']
for feature in cat_features:
    train_df[feature] = train_df[feature].astype('category')
    print(feature,train_df[feature].dtype,train_df[feature].nunique())

gender category 4
signup_method category 3
language category 25
affiliate_channel category 8
affiliate_provider category 18
first_affiliate_tracked category 7
signup_app category 4
first_device_type category 9
first_browser category 52
country_destination category 12


In [43]:
for feature in cat_features:
    print(feature)
    temp_df = pd.get_dummies(train_df[feature],prefix=feature)
    train_df = pd.concat([train_df,temp_df],axis=1)

gender
signup_method
language
affiliate_channel
affiliate_provider
first_affiliate_tracked
signup_app
first_device_type
first_browser
country_destination


In [44]:
train_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,country_destination_DE,country_destination_ES,country_destination_FR,country_destination_GB,country_destination_IT,country_destination_NDF,country_destination_NL,country_destination_PT,country_destination_US,country_destination_other
0,gxn3p5htnn,2010-06-28,-1537967433,NaN,-unknown-,34.0,facebook,0,en,direct,...,0,0,0,0,0,1,0,0,0,0
1,820tgsjxq7,2011-05-25,-1333835879,NaN,MALE,38.0,facebook,0,en,seo,...,0,0,0,0,0,1,0,0,0,0
2,4ft3gnwmtx,2010-09-28,-1247779441,2010-08-02,FEMALE,56.0,basic,3,en,direct,...,0,0,0,0,0,0,0,0,1,0
3,bjjt8pjhuk,2011-12-05,-825950559,2012-09-08,FEMALE,42.0,facebook,0,en,direct,...,0,0,0,0,0,0,0,0,0,1
4,87mebub9p4,2010-09-14,-648949583,2010-02-18,-unknown-,41.0,basic,0,en,direct,...,0,0,0,0,0,0,0,0,1,0
